In [1]:
example = """
nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
""".strip().splitlines()

with open("day08.txt", "r") as f:
    data = f.readlines()

In [25]:
class State(object):
    def __init__(self):
        self.accumulator = 0
        self.program_counter = 0

    def accumulate(self, add: int):
        self.accumulator += add

    def jump(self, offset: int):
        self.program_counter += offset - 1

    def reset(self):
        self.accumulator = 0
        self.program_counter = 0

In [26]:
from typing import Dict, Callable
from re import match

class Instruction(object):
    def __init__(self, instruction: str):
        matches = match(r"^(\w+) ([+\-]\d+)$", instruction)
        self.code = matches[1]
        self.argument = int(matches[2])

        self.run_count = 0

    kinds: Dict[str, Callable[[int, State], None]] = {
        "nop": lambda arg, state: 0,
        "acc": lambda arg, state: state.accumulate(arg),
        "jmp": lambda arg, state: state.jump(arg),
    }

    def run(self, state: State):
        if self.code in Instruction.kinds:
            Instruction.kinds[self.code](self.argument, state)
        
        self.run_count += 1
        state.program_counter += 1

    def reset(self):
        self.run_count = 0

assert Instruction("nop +0").code == "nop"
assert Instruction("nop +0").argument == 0

test_state = State()
test_instruction = Instruction("acc +5")
test_instruction.run(test_state)
assert test_instruction.run_count == 1
assert test_state.accumulator == 5
assert test_state.program_counter == 1

In [27]:
from typing import List

class Program(object):
    def __init__(self, instructions: List[str]):
        self.instructions = [Instruction(instruction) for instruction in instructions]
        self.state = State()

    def run_distinct(self):
        while 0 <= self.state.program_counter < len(self.instructions) and self.instructions[self.state.program_counter].run_count < 1:
            self.instructions[self.state.program_counter].run(self.state)

    def run_to_end(self) -> bool:
        while 0 <= self.state.program_counter < len(self.instructions) and self.instructions[self.state.program_counter].run_count < 1:
            self.instructions[self.state.program_counter].run(self.state)

        return self.state.program_counter < 0 or self.state.program_counter >= len(self.instructions)

    def reset(self):
        self.state.reset()
        for instruction in self.instructions:
            instruction.reset()

test_program = Program(example)
test_program.run_distinct()
assert test_program.state.accumulator == 5

In [28]:
program = Program(data)
program.run_distinct()
print(f"Program accumulator value was {program.state.accumulator} before it looped")

Program accumulator value was 1810 before it looped


## Second Part
We'll iterate through each instruction and test, if we switch it from `nop` to `jmp` (or vice versa) whether the program terminates correctly.

In [31]:
for instruction in program.instructions:
    program.reset()
    old_code = instruction.code

    if old_code == "nop":
        instruction.code = "jmp"
    elif old_code == "jmp":
        instruction.code = "nop"
    else:
        # We know that making a change will not alter the outcome
        continue

    if program.run_to_end():
        print(f"Found a termination condition with accumulator {program.state.accumulator}")

    instruction.code = old_code

Found a termination condition with accumulator 969
